In [1]:
import psycopg2
import pandas as pd

In [2]:


# params PostgreSQL
db_params = {
    'dbname': 'DBInmerso',
    'user': 'postgres',
    'password': 'postgres',
    'host': 'localhost',
    'port': '5432'
}

# table

tables_to_save = ['usuarios', 'discapacidad', 'renta', 'patrimonio', 'familia_numerosa', 'disponibilidad', 'historial_usuarios', 'tipo_resolucion', 'solicitudes', 'hoteles', 'destinos', 'resolucion_solicitudes', 'scoring']


# Connection
connection = psycopg2.connect(**db_params)

# Cursor
cursor = connection.cursor()


for table in tables_to_save:
    # DataFrame
    sql_query = """
        SELECT
            u.dni,
            u.nombre,
            u.apellido,
            u.edad,
            u.fecha_de_nacimiento,
            u.id_solicitud,
            u.usuario_solicitante,
            u.oficio_especial,
            r.ingresos,
            d.grado_dis,
            fn.tipo_fam_num,
            p.valoracion_patrimonio,
            p.coche,
            p.arrendador,
            s.anyo_solicitud,
            s.usuarios_sol,
            s.renta_sol,
            s.primera_opcion,
            s.fecha_1op,
            s.segunda_opcion,
            s.fecha_2op,
            s.tercera_opcion,
            s.fecha_3op

        FROM
            usuarios u
        JOIN renta r ON u.dni = r.dni
        JOIN discapacidad d ON u.dni = d.dni
        JOIN familia_numerosa fn ON u.dni = fn.dni
        JOIN patrimonio p ON u.dni = p.dni
        JOIN solicitudes s ON u.id_solicitud = s.id_solicitud;
    """
    df = pd.read_sql_query(sql_query, connection)



# Close
cursor.close()
connection.close()




/var/folders/gb/qf4chfw11j5dsg0zrngx5sy00000gn/T/ipykernel_5964/3349861315.py:58: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(sql_query, connection)


In [3]:
pd.set_option('display.max_columns', None)

In [4]:
df.head(5)

,dni,nombre,apellido,edad,fecha_de_nacimiento,id_solicitud,usuario_solicitante,oficio_especial,ingresos,grado_dis,tipo_fam_num,valoracion_patrimonio,coche,arrendador,anyo_solicitud,usuarios_sol,renta_sol,primera_opcion,fecha_1op,segunda_opcion,fecha_2op,tercera_opcion,fecha_3op
0,95822412,Natalia,Belda,62,1936-10-04,52,False,False,2336.0,2,1,2000000.0,False,True,2024,0.0,0,Tarragona,2024-12-06,Toledo,2024-11-20,Melilla,2024-04-29
1,46913810,Francisco Javier,Mendizábal,70,1917-11-13,458,True,False,4247.0,2,2,441843.0,False,False,2024,0.0,0,Cuenca,2024-04-07,Soria,2024-02-01,Zamora,2024-02-24
2,28728463,Cecilia,Tudela,61,1911-12-18,1386,False,True,3132.0,0,0,138429.0,True,True,2024,0.0,0,Cuenca,2024-05-15,Cuenca,2024-07-25,Girona,2024-07-22
3,83197857,Julio,Navas,60,1937-02-02,1210,False,False,3452.0,1,0,347455.0,False,False,2024,0.0,0,Girona,2024-11-15,Sevilla,2024-05-11,Huesca,2024-12-21
4,22575562,Yago,Cortés,68,1937-04-02,477,True,False,4873.0,0,2,68212.0,True,False,2024,0.0,0,Huesca,2024-02-24,Pontevedra,2024-09-04,Melilla,2024-05-09


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 992 entries, 0 to 991
Data columns (total 23 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   dni                    992 non-null    object 
 1   nombre                 992 non-null    object 
 2   apellido               992 non-null    object 
 3   edad                   992 non-null    int64  
 4   fecha_de_nacimiento    992 non-null    object 
 5   id_solicitud           992 non-null    int64  
 6   usuario_solicitante    992 non-null    bool   
 7   oficio_especial        992 non-null    bool   
 8   ingresos               992 non-null    float64
 9   grado_dis              992 non-null    int64  
 10  tipo_fam_num           992 non-null    int64  
 11  valoracion_patrimonio  992 non-null    float64
 12  coche                  992 non-null    bool   
 13  arrendador             992 non-null    bool   
 14  anyo_solicitud         992 non-null    int64  
 15  usuari

In [6]:
# SCORE EDAD


# add score

for index, row in df.iterrows():
    edad = row['edad']
    SCORE = 0
    if edad < 60:
        SCORE = 1
    elif edad == 60:
        SCORE = 2
    elif edad <= 78:
        SCORE = 2 + (edad - 60)
    else:
        SCORE = 20
    
    df.at[index, 'score_edad'] = SCORE

df['score_edad'] = df['score_edad'].astype(int)


df.head(5)

,dni,nombre,apellido,edad,fecha_de_nacimiento,id_solicitud,usuario_solicitante,oficio_especial,ingresos,grado_dis,tipo_fam_num,valoracion_patrimonio,coche,arrendador,anyo_solicitud,usuarios_sol,renta_sol,primera_opcion,fecha_1op,segunda_opcion,fecha_2op,tercera_opcion,fecha_3op,score_edad
0,95822412,Natalia,Belda,62,1936-10-04,52,False,False,2336.0,2,1,2000000.0,False,True,2024,0.0,0,Tarragona,2024-12-06,Toledo,2024-11-20,Melilla,2024-04-29,4
1,46913810,Francisco Javier,Mendizábal,70,1917-11-13,458,True,False,4247.0,2,2,441843.0,False,False,2024,0.0,0,Cuenca,2024-04-07,Soria,2024-02-01,Zamora,2024-02-24,12
2,28728463,Cecilia,Tudela,61,1911-12-18,1386,False,True,3132.0,0,0,138429.0,True,True,2024,0.0,0,Cuenca,2024-05-15,Cuenca,2024-07-25,Girona,2024-07-22,3
3,83197857,Julio,Navas,60,1937-02-02,1210,False,False,3452.0,1,0,347455.0,False,False,2024,0.0,0,Girona,2024-11-15,Sevilla,2024-05-11,Huesca,2024-12-21,2
4,22575562,Yago,Cortés,68,1937-04-02,477,True,False,4873.0,0,2,68212.0,True,False,2024,0.0,0,Huesca,2024-02-24,Pontevedra,2024-09-04,Melilla,2024-05-09,10


In [7]:
#SCORE discapacidad


# add score
for index, row in df.iterrows():
    grado_dis = row['grado_dis']
    SCORE = 0
    if grado_dis == 1:
        SCORE = 10
    else:
        grado_dis == 2
        SCORE = 15

    
    df.at[index, 'score_discapacidad'] = SCORE

df['score_discapacidad'] = df['score_discapacidad'].astype(int)

df.head(5)

,dni,nombre,apellido,edad,fecha_de_nacimiento,id_solicitud,usuario_solicitante,oficio_especial,ingresos,grado_dis,tipo_fam_num,valoracion_patrimonio,coche,arrendador,anyo_solicitud,usuarios_sol,renta_sol,primera_opcion,fecha_1op,segunda_opcion,fecha_2op,tercera_opcion,fecha_3op,score_edad,score_discapacidad
0,95822412,Natalia,Belda,62,1936-10-04,52,False,False,2336.0,2,1,2000000.0,False,True,2024,0.0,0,Tarragona,2024-12-06,Toledo,2024-11-20,Melilla,2024-04-29,4,15
1,46913810,Francisco Javier,Mendizábal,70,1917-11-13,458,True,False,4247.0,2,2,441843.0,False,False,2024,0.0,0,Cuenca,2024-04-07,Soria,2024-02-01,Zamora,2024-02-24,12,15
2,28728463,Cecilia,Tudela,61,1911-12-18,1386,False,True,3132.0,0,0,138429.0,True,True,2024,0.0,0,Cuenca,2024-05-15,Cuenca,2024-07-25,Girona,2024-07-22,3,15
3,83197857,Julio,Navas,60,1937-02-02,1210,False,False,3452.0,1,0,347455.0,False,False,2024,0.0,0,Girona,2024-11-15,Sevilla,2024-05-11,Huesca,2024-12-21,2,10
4,22575562,Yago,Cortés,68,1937-04-02,477,True,False,4873.0,0,2,68212.0,True,False,2024,0.0,0,Huesca,2024-02-24,Pontevedra,2024-09-04,Melilla,2024-05-09,10,15


In [10]:
df['usuarios_sol'] = df['usuarios_sol'].astype(float)

In [18]:
# SCORE RENTA


# add score

for usuarios_sol in df['usuarios_sol']:
    if usuarios_sol > 1:
        for index, row in df.iterrows():
            renta_sol = row['renta_sol']
            
            if renta_sol <= 484.61:
                SCORE = 50
            elif 484.61 < renta_sol <= 900:
                SCORE = 45
            elif 900 < renta_sol <= 1050:
                SCORE = 40
            elif 1050 < renta_sol <= 1200:
                SCORE = 35
            elif 1200 < renta_sol <= 1350:
                SCORE = 30
            elif 1350 < renta_sol <= 1500:
                SCORE = 25
            elif 1500 < renta_sol <= 1650:
                SCORE = 20
            elif 1650 < renta_sol <= 1800:
                SCORE = 15
            elif 1800 < renta_sol <= 1950:
                SCORE = 10
            elif 1950 < renta_sol <= 2100:
                SCORE = 5
            else:
                SCORE = 0
    else:
        for index, row in df.iterrows():
            ingresos = row['ingresos']
            
            if ingresos <= 484.61:
                SCORE = 50
            elif 484.61 < ingresos <= 900:
                SCORE = 45
            elif 900 < ingresos <= 1050:
                SCORE = 40
            elif 1050 < ingresos <= 1200:
                SCORE = 35
            elif 1200 < ingresos <= 1350:
                SCORE = 30
            elif 1350 < ingresos <= 1500:
                SCORE = 25
            elif 1500 < ingresos <= 1650:
                SCORE = 20
            elif 1650 < ingresos <= 1800:
                SCORE = 15
            elif 1800 < ingresos <= 1950:
                SCORE = 10
            elif 1950 < ingresos <= 2100:
                SCORE = 5
            else:
                SCORE = 0
        


    df.at[index, 'score_renta'] = SCORE

df['score_renta'] = df['score_renta'].astype(float)

df.head(5)




,dni,nombre,apellido,edad,fecha_de_nacimiento,id_solicitud,usuario_solicitante,oficio_especial,ingresos,grado_dis,tipo_fam_num,valoracion_patrimonio,coche,arrendador,anyo_solicitud,usuarios_sol,renta_sol,primera_opcion,fecha_1op,segunda_opcion,fecha_2op,tercera_opcion,fecha_3op,score_edad,score_discapacidad,score_renta
0,95822412,Natalia,Belda,62,1936-10-04,52,False,False,2336.0,2,1,2000000.0,False,True,2024,0.0,0,Tarragona,2024-12-06,Toledo,2024-11-20,Melilla,2024-04-29,4,15,NaN
1,46913810,Francisco Javier,Mendizábal,70,1917-11-13,458,True,False,4247.0,2,2,441843.0,False,False,2024,0.0,0,Cuenca,2024-04-07,Soria,2024-02-01,Zamora,2024-02-24,12,15,NaN
2,28728463,Cecilia,Tudela,61,1911-12-18,1386,False,True,3132.0,0,0,138429.0,True,True,2024,0.0,0,Cuenca,2024-05-15,Cuenca,2024-07-25,Girona,2024-07-22,3,15,NaN
3,83197857,Julio,Navas,60,1937-02-02,1210,False,False,3452.0,1,0,347455.0,False,False,2024,0.0,0,Girona,2024-11-15,Sevilla,2024-05-11,Huesca,2024-12-21,2,10,NaN
4,22575562,Yago,Cortés,68,1937-04-02,477,True,False,4873.0,0,2,68212.0,True,False,2024,0.0,0,Huesca,2024-02-24,Pontevedra,2024-09-04,Melilla,2024-05-09,10,15,NaN


In [ ]:
# SCORE patrimonio

# add score

for usuarios_sol in df['usuarios_sol']:
    if usuarios_sol > 1:
        for index, row in df.iterrows():
            patrimonio_sol = row['patrimonio_sol']
            SCORE = 0
            if patrimonio_sol <= 20000:
                SCORE = 150
            elif 20000 < patrimonio_sol <= 40000:
                SCORE = 100
            elif 40000 < patrimonio_sol <= 80000:
                SCORE = 55
            elif 80000 < patrimonio_sol <= 130000:
                SCORE = 45
            elif 130000 < patrimonio_sol <= 170000:
                SCORE = 35
            elif 170000 < patrimonio_sol <= 230000:
                SCORE = 25
            elif 230000 < patrimonio_sol <= 275000:
                SCORE = 15
            elif 275000 < patrimonio_sol <= 325000:
                SCORE = 5
            else:
                SCORE = 0
    else:
        for index, row in df.iterrows():
            patrimonio = row['valoracion_patrimonio']
            SCORE = 0
            if patrimonio <= 20000:
                SCORE = 150
            elif 20000 < patrimonio <= 40000:
                SCORE = 100
            elif 40000 < patrimonio <= 80000:
                SCORE = 55
            elif 80000 < patrimonio <= 130000:
                SCORE = 45
            elif 130000 < patrimonio <= 170000:
                SCORE = 35
            elif 170000 < patrimonio <= 230000:
                SCORE = 25
            elif 230000 < patrimonio <= 275000:
                SCORE = 15
            elif 275000 < patrimonio <= 325000:
                SCORE = 5
            else:
                SCORE = 0
        


    df.at[index, 'score_patrimonio'] = SCORE

df['score_patrimonio'] = df['score_patrimonio'].astype(int)

df.head(5)

In [22]:
# SCORE FAMILIA NUMEROSA



# add score
for index, row in df.iterrows():
    tipo_fam_num = row['tipo_fam_num']
    SCORE = 0
    if tipo_fam_num == 1:
        SCORE = 5
    elif tipo_fam_num == 2:
        SCORE = 10

    
    df.at[index, 'score_fam_num'] = SCORE

df['score_fam_num'] = df['score_fam_num'].astype(int)

df.head(5)




,dni,nombre,apellido,edad,fecha_de_nacimiento,id_solicitud,usuario_solicitante,oficio_especial,ingresos,grado_dis,tipo_fam_num,valoracion_patrimonio,coche,arrendador,anyo_solicitud,usuarios_sol,renta_sol,primera_opcion,fecha_1op,segunda_opcion,fecha_2op,tercera_opcion,fecha_3op,score_edad,score_discapacidad,score_renta,score_fam_num
0,95822412,Natalia,Belda,62,1936-10-04,52,False,False,2336.0,2,1,2000000.0,False,True,2024,0.0,0,Tarragona,2024-12-06,Toledo,2024-11-20,Melilla,2024-04-29,4,15,NaN,5
1,46913810,Francisco Javier,Mendizábal,70,1917-11-13,458,True,False,4247.0,2,2,441843.0,False,False,2024,0.0,0,Cuenca,2024-04-07,Soria,2024-02-01,Zamora,2024-02-24,12,15,NaN,10
2,28728463,Cecilia,Tudela,61,1911-12-18,1386,False,True,3132.0,0,0,138429.0,True,True,2024,0.0,0,Cuenca,2024-05-15,Cuenca,2024-07-25,Girona,2024-07-22,3,15,NaN,0
3,83197857,Julio,Navas,60,1937-02-02,1210,False,False,3452.0,1,0,347455.0,False,False,2024,0.0,0,Girona,2024-11-15,Sevilla,2024-05-11,Huesca,2024-12-21,2,10,NaN,0
4,22575562,Yago,Cortés,68,1937-04-02,477,True,False,4873.0,0,2,68212.0,True,False,2024,0.0,0,Huesca,2024-02-24,Pontevedra,2024-09-04,Melilla,2024-05-09,10,15,NaN,10


In [ ]:
# coche oficio arrendador